In [5]:
# pandas-datareader
# scikit-learn
# tensorflow
# yfinance

In [ ]:
# %pip install yfinance


/Users/hanif/Documents/machine learning/ann/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[*********************100%%**********************]  1 of 1 completed

                    Open          High           Low         Close  \
Date                                                                 
2021-10-12  57526.832031  57627.878906  54477.972656  56041.058594   
2021-10-13  56038.257812  57688.660156  54370.972656  57401.097656   
2021-10-14  57372.832031  58478.734375  56957.074219  57321.523438   
2021-10-15  57345.902344  62757.128906  56868.144531  61593.949219   
2021-10-16  61609.527344  62274.476562  60206.121094  60892.179688   
...                  ...           ...           ...           ...   
2023-12-14  42884.261719  43390.859375  41767.089844  43023.972656   
2023-12-15  43028.250000  43087.824219  41692.968750  41929.757812   
2023-12-16  41937.742188  42664.945312  41723.113281  42240.117188   
2023-12-17  42236.109375  42359.496094  41274.542969  41364.664062   
2023-12-18  41348.203125  42720.296875  40530.257812  42623.539062   

               Adj Close       Volume  
Date                                   
2021-10-1

In [5]:
data.shape

(798, 6)

In [4]:
data.to_csv("btcprice.csv")

In [17]:
# Scale data 
scaler = MinMaxScaler(feature_range=(0,1))   # Scale data between 0 and 1

scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

predict_days = 60

x_train, y_train = [],[]

for x in range(predict_days, len(scaled_data)):
     x_train.append(scaled_data[x-predict_days:x, 0])
     y_train.append(scaled_data[x, 0])
     
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))



In [ ]:
# Create Neural Network
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

In [ ]:
# Test data
test_start = dt.datetime(2020, 1, 1)
test_end = dt.datetime.now()

test_data = yf.download(f'{crypto_currency}-{against_currency}', start=test_start, end=test_end)

actual_prices = test_data['Close'].values

total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_data) - predict_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.fit_transform(model_inputs)

In [ ]:
x_test = []
for x in range(predict_days, len(model_inputs)):
     x_test.append(model_inputs[x-predict_days:x, 0])
     
     
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
# Prediction prices

prediction_prices = model.predict(x_test)
prediction_prices=scaler.inverse_transform(prediction_prices)

In [ ]:
# Plot prices
plt.plot(actual_prices, color='black', label=f'Actual {crypto_currency} Price')
plt.plot(prediction_prices, color='green', label=f'Predicted {crypto_currency} Price')
plt.title(f'{crypto_currency} Price Prediction')
plt.xlabel('Time')
plt.ylabel(f'{crypto_currency} Price')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Predict next day

real_data = [model_inputs[len(model_inputs) + 1 - predict_days:len(model_inputs) + 1, 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f'Predicted {crypto_currency} price for next day: {prediction}')